In [7]:
import nltk
from nltk.corpus import stopwords
import unidecode
# Télécharger les stop words (à faire une fois)
# nltk.download('stopwords')
import re
def clean_sentence(sentence):
    # Votre liste initiale de mots à exclure
    custom_exclude_words = {
        'et', 'des', 'les', 'de', 'la', 'le', 'en', 'du', 'pour', 'comment',
        'avec', 'sans', 'sous', 'par', 'dans', 'sur', 'entre', 'contre', 'vers',
        'après', 'avant', 'chez', 'pendant', 'depuis', 'jusque', 'jusqu', 'tandis',
        'que', 'comme', 'si', 'lorsque', 'puisque', 'quoique', 'bien', 'ainsi'
    }

    # Stop words en français et en anglais
    stop_words_fr = set(stopwords.words('french'))
    stop_words_en = set(stopwords.words('english'))
    exclude_words = custom_exclude_words.union(stop_words_fr, stop_words_en)

    # Remplacer les caractères accentués
    sentence = unidecode.unidecode(sentence)

    # Supprimer le caractère avant les guillemets doubles
    sentence = re.sub(r".''", '', sentence)

    # Convertir en minuscules, supprimer les caractères spéciaux (sauf les tirets, les barres obliques)
    sentence = re.sub(r'[^A-Za-z\s/-]', '', sentence.lower())

    # Split sur les espaces, les tirets, et les barres obliques
    words = re.split(r'[\s/-]+', sentence)

    # Filtrer les mots, les mettre au singulier si nécessaire et exclure les mots d'une lettre
    filtered_words = [word for word in words if word not in exclude_words and len(word) > 1]

    return ", ".join(filtered_words)


# Exemple d'utilisation
sentence = "Commercial - Ventes"
cleaned_name = clean_sentence(sentence)
print(cleaned_name)  # Devrait afficher une liste de mots clés nettoyés

commercial, ventes


In [13]:
import cx_Oracle
#from spellchecker import SpellChecker

# Créer un objet SpellChecker pour le français
#spell = SpellChecker(language='fr')

# Paramètres de connexion
username = "SYS"
password = "2000"
hostname = "localhost"
port = "1521"
sid = "xe"

# Création de la chaîne DSN
dsn = cx_Oracle.makedsn(hostname, port, sid=sid)

# Connexion à la base de données
try:
    connection = cx_Oracle.connect(username, password, dsn, mode=cx_Oracle.SYSDBA)
    print("Connexion réussie à la base de données Oracle")

    # Création d'un curseur
    cursor = connection.cursor()

    # Activer DBMS_OUTPUT
    cursor.callproc("dbms_output.enable")



    # Keyword original
    keyword = 'power bi'
 
    corrected_keyword = clean_sentence(keyword)
    print(corrected_keyword)
    cursor.execute("""
DECLARE

    c SYS_REFCURSOR;
    result_row VARCHAR2(4000);
BEGIN
    c := search_phrase('SOUSSOUSDomaineFormation', 'Motscles', :keyword);

    LOOP
        FETCH c INTO result_row;
        EXIT WHEN c%NOTFOUND;
        DBMS_OUTPUT.PUT_LINE(result_row);
    END LOOP;

    CLOSE c;
END;

    """, keyword=corrected_keyword)

    # Récupérer et afficher les lignes de DBMS_OUTPUT
    statusVar = cursor.var(cx_Oracle.NUMBER)
    lineVar = cursor.var(cx_Oracle.STRING)
    while True:
        cursor.callproc("dbms_output.get_line", (lineVar, statusVar))
        if statusVar.getvalue() != 0:
            break
        print(lineVar.getvalue())

    # Fermeture du curseur
    cursor.close()

except cx_Oracle.DatabaseError as e:
    print("Erreur lors de la connexion à la base de données", e)

finally:
    # Fermeture de la connexion
    if connection:
        connection.close()
        print("Connexion à la base de données fermée")


Connexion réussie à la base de données Oracle
power, bi
power, bi, desktop, analyse, donnees, tableaux, bord, best
power, bi, avance, new
Connexion à la base de données fermée
